## 課題提出時（2018/11/26）の所感
sprint11では初めてTensorFlowのコードリーディングを行い、さわってみました。確かにスクラッチよりもずっと簡単にNNを組めるので大変便利だと感じました。まだ使い方に慣れていないので、おいおい使って行きたいと思います。

# 1.この課題の目的
フレームワークのコードを読めるようにする  
フレームワークを習得し続けられるようになる  
理論を知っている範囲をフレームワークで動かす  

【目的としないこと】  
各フレームワークの細かいノウハウの習得  
デプロイなど動かしてみたその先  
新たに見かけた理論の詳細な理解  

# 2.進め方
コードリーディング（1日目）  
いくつかのデータセットをフレームワークで扱う（1日目〜2日目）  

# 3.コードリーディング
TensorFLowによって2値分類を行うサンプルコードを載せた。今回はこれをベースにして進める。
tf.kerasやtf.estimatorなどの高レベルAPIは使用していない。既にNumPyによるスクラッチを行っているから、フレームワークも低レベルなところから見ていくことにする。



## スクラッチを振り返る
sprint9,10で行ったスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙する。

・トレイナークラス・・・Fit/Predict  
・最適化クラス（Optimizer）・・・SGD  
・活性化関数クラス（Activationfunction） ・・・Tanh,sigmoid <backward>  
・損失関数クラス（Metrics）（測定基準）：MSE/交差エントロピー <backward>  
・全結合層クラス・・・np.dot  、<backward>  
・初期化クラス・・・重み・バイアス  
・エポック、ミニバッチ  
・前処理  
    
    
 それらがフレームワークにおいてはどのように実装されるかを確認していく。
 
 【調査結果】
NumPy  ：  np.dot
TensorFlow  :  tf.matmul

## データセットの用意

In [13]:
#データの取り込み
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import  train_test_split

%matplotlib inline
# Irisデータセットの用意
iris = pd.read_csv("/Users/tsuneo/kaggle/IrisSpecies/Iris.csv")
# 型の確認
iris.shape

(150, 6)

In [14]:
iris.head(3)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa


In [15]:
#  目的変数Speciesの中から2つに限定する
iris = iris.query('Species in ["Iris-versicolor", "Iris-virginica"]')

In [16]:
iris.shape

(100, 6)

In [17]:
# irisのデータを説明変数、目的変数に分割する
# X = iris[["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]].values
# y = iris[["Species"]]
# print(type(y))
# y["Species"] = y["Species"].map({'Iris-versicolor':0, 'Iris-virginica':1})[:,None]
# print(type(y))
# X.shape, y.shape

## サンプルコード
以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認する。
それを簡単に言葉でまとめておくこと。単純な一対一の対応であるとは限らない。

＊バージョン1.5から1.12の間では動作を確認済みである。

In [63]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類するコード
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

# データセットの読み込み
dataset_path ="/Users/tsuneo/kaggle/IrisSpecies/Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._counter = 0
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __iter__(self):
        return self
    def __next__(self):
        if self._counter >= self._stop:
            self._counter = 0
            raise StopIteration()

        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    # 全結合層っぽい
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    
    # 活性化関数reluのようだ
    layer_1 = tf.nn.relu(layer_1)
    
    # 全結合
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    
    # 活性化関数relu
    layer_2 = tf.nn.relu(layer_2)
    
    # 全結合のままで出力しているので、恒等関数で対応している意味合い？
    # →違う。出力層の活性化関数はsigmoidである。誤差逆伝搬のコードは★１
    # 誤差逆伝搬ではない、予測値を算出する処理は★2
    # つまり、誤差逆伝搬の「計算グラフ」では、★1と★２に枝分かれしている
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
# reduce_meanは、np.meanと等価のようだ
# シグモイド＋交差エントロピーの組み合わせ.サンプル毎の損失をだすため、reduce_meanしている
# ★１
# ◼️◼️◼️◼️◼️ループで呼び出している箇所
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))

# 最適化手法
# AdamOptimizer　：Adamアルゴリズムを実装するオプティマイザ
# 重みを更新する処理
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

# ◼️◼️◼️◼️◼️ループで呼び出している箇所
# loss更新によって最小化する操作を追加し、var_list
train_op = optimizer.minimize(loss_op)

# 推定結果 -0.5は閾値　★2
tmp1 = tf.sign(Y - 0.5) # debug
tmp2 = tf.sigmoid(logits)
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))

# 指標値計算
# ◼️◼️◼️◼️◼️ループで呼び出している箇所
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    # このrunでは、initだけを実行している。
    sess.run(init)
    for epoch in range(num_epochs):
        
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        #print(X_train.shape[])
        total_loss = 0
        total_acc = 0
        
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            
            # ミニバッチごとにループ
            # 
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            #aa  = sess.run(correct_pred, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            #bb = sess.run(correct_pred, feed_dict={X: mini_batch_x, Y: mini_batch_y}) #debug
            #print(acc)
            #cc = sess.run(tmp2, feed_dict={X: mini_batch_x, Y: mini_batch_y})  #debug
            #print("correct_pred", bb.dtype)
            #print("tf.sigmoid(logits).shape",cc.shape)
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))
    

Epoch 0, loss : 1.7807, val_loss : 7.4728, acc : 0.750, val_acc : 0.438
Epoch 1, loss : 0.0000, val_loss : 0.8265, acc : 1.000, val_acc : 0.812
Epoch 2, loss : 0.0014, val_loss : 4.3522, acc : 1.000, val_acc : 0.750
Epoch 3, loss : 1.5344, val_loss : 7.7090, acc : 0.750, val_acc : 0.500
Epoch 4, loss : 0.1619, val_loss : 6.5967, acc : 1.000, val_acc : 0.750
Epoch 5, loss : 2.0780, val_loss : 8.6058, acc : 0.750, val_acc : 0.500
Epoch 6, loss : 0.0013, val_loss : 5.6772, acc : 1.000, val_acc : 0.750
Epoch 7, loss : 1.5678, val_loss : 7.9787, acc : 0.750, val_acc : 0.500
Epoch 8, loss : 0.0000, val_loss : 3.9957, acc : 1.000, val_acc : 0.812
Epoch 9, loss : 0.2258, val_loss : 6.7578, acc : 0.750, val_acc : 0.562
test_acc : 0.700


In [50]:
bb.shape

(4, 1)

# 4.他のデータセットへの適用
これまでに扱ってきた小さなデータセットがいくつかある。これらに対して学習・推定を行うニューラルネットワークを作成する。
どれも簡単なデータセットではあるが、新しいデータに対してフレームワークを動かすということは今後の基本になる部分である。  
Iris（3種類全ての目的変数を使用）  
MNIST  
house price  
どのデータセットもtrain, val, testの3種類に分けて使うこと。

# ---------Iris（3種類全ての目的変数を使用）---------
ポイントはone-hotにしないといけない。  
２値分類は出力層が１つにできるが、３分類からはone-hotにしなければいけない

In [54]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを3値分類するコード
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

# データセットの読み込み
dataset_path ="/Users/tsuneo/kaggle/IrisSpecies/Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
#df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y[y=='Iris-setosa'] = 2
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

(96, 4) (24, 4) (30, 4)
(96, 1) (24, 1) (30, 1)


In [55]:
#  one-hot表現にする
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

# 教師ラベルだけ変換する
#y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
#y_test_one_hot = enc.transform(y_test[:, np.newaxis])
#y_val_one_hot = enc.transform(y_val[:, np.newaxis])
y_train_one_hot = enc.fit_transform(y_train)
y_test_one_hot = enc.transform(y_test)
y_val_one_hot = enc.transform(y_val)

# shapeの確認
print("y_train.shape    　　　　　　　　　　　　　　　　:   ",y_train.shape)
print("y_train_one_hot.shape    :   ",y_train_one_hot.shape)
print("y_train_one_hot.型  　　　　　　  :   ",y_train_one_hot.dtype)
print("------")
print("y_test.shape  　　　　　　　　　　　　　　　　  :   ",y_test.shape)
print("y_test_one_hot.shape    :   ",y_test_one_hot.shape)
print("y_test_one_hot.型　　　　　　    :   ",y_test_one_hot.dtype)
print("------")
print("y_val.shape  　　　　　　　　　　　　　　　　  :   ",y_val.shape)
print("y_val_one_hot.shape    :   ",y_val_one_hot.shape)
print("y_val_one_hot.型　　　　　　    :   ",y_val_one_hot.dtype)

y_train.shape    　　　　　　　　　　　　　　　　:    (96, 1)
y_train_one_hot.shape    :    (96, 3)
y_train_one_hot.型  　　　　　　  :    float64
------
y_test.shape  　　　　　　　　　　　　　　　　  :    (30, 1)
y_test_one_hot.shape    :    (30, 3)
y_test_one_hot.型　　　　　　    :    float64
------
y_val.shape  　　　　　　　　　　　　　　　　  :    (24, 1)
y_val_one_hot.shape    :    (24, 3)
y_val_one_hot.型　　　　　　    :    float64


In [68]:
class GetMiniBatch2:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._counter = 0
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __iter__(self):
        return self
    def __next__(self):
        if self._counter >= self._stop:
            self._counter = 0
            raise StopIteration()

        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.0002
batch_size = 12
num_epochs = 20

n_hidden1 = 300
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch2(X_train, y_train_one_hot, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    # 全結合層っぽい
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    
    # 活性化関数reluのようだ
    layer_1 = tf.nn.relu(layer_1)
    
    # 全結合
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    
    # 活性化関数relu
    layer_2 = tf.nn.relu(layer_2)
    
    # 全結合のままで出力しているので、恒等関数で対応している意味合い？
    # →違う。出力層の活性化関数はsigmoidである。誤差逆伝搬のコードは★１
    # 誤差逆伝搬ではない、予測値を算出する処理は★2
    # つまり、誤差逆伝搬の「計算グラフ」では、★1と★２に枝分かれしている
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
# reduce_meanは、np.meanと等価のようだ
# シグモイド＋交差エントロピーの組み合わせ.サンプル毎の損失をだすため、reduce_meanしている
# ★１
# ◼️◼️◼️◼️◼️ループで呼び出している箇所
#loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))

# 最適化手法
# AdamOptimizer　：Adamアルゴリズムを実装するオプティマイザ
# 重みを更新する処理
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

# ◼️◼️◼️◼️◼️ループで呼び出している箇所
# loss更新によって最小化する操作を追加し、var_list
train_op = optimizer.minimize(loss_op)

# 推定結果 -0.5は閾値　★2
#tmp1 = tf.sign(Y - 0.5) # debug
#tmp2 = tf.sigmoid(logits)
#correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
#correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.nn.softmax(logits) - 0.5))
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(logits, 1))

# 指標値計算
# ◼️◼️◼️◼️◼️ループで呼び出している箇所
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    # このrunでは、initだけを実行している。
    sess.run(init)
    for epoch in range(num_epochs):
        
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        #print(X_train.shape[])
        total_loss = 0
        total_acc = 0
        
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            
            # ミニバッチごとにループ
            # 
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            #aa  = sess.run(correct_pred, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            #bb = sess.run(correct_pred, feed_dict={X: mini_batch_x, Y: mini_batch_y}) #debug
            #print(acc)
            #cc = sess.run(tmp2, feed_dict={X: mini_batch_x, Y: mini_batch_y})  #debug
            #print("correct_pred", bb.dtype)
            #print("tf.sigmoid(logits).shape",cc.shape)
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val_one_hot})
        print("Epoch {}, loss : {:.2f}, val_loss : {:.2f}, acc : {:.2f}, val_acc : {:.2f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test_one_hot})
    print("test_acc : {:.2f}".format(test_acc))
    

Epoch 0, loss : 377.51, val_loss : 333.63, acc : 0.67, val_acc : 0.71
Epoch 1, loss : 338.24, val_loss : 297.11, acc : 0.67, val_acc : 0.71
Epoch 2, loss : 298.80, val_loss : 260.48, acc : 0.67, val_acc : 0.71
Epoch 3, loss : 259.36, val_loss : 223.79, acc : 0.67, val_acc : 0.71
Epoch 4, loss : 220.02, val_loss : 187.04, acc : 0.67, val_acc : 0.71
Epoch 5, loss : 180.73, val_loss : 150.28, acc : 0.67, val_acc : 0.71
Epoch 6, loss : 141.52, val_loss : 113.46, acc : 0.67, val_acc : 0.71
Epoch 7, loss : 102.23, val_loss : 76.55, acc : 0.67, val_acc : 0.71
Epoch 8, loss : 62.85, val_loss : 39.60, acc : 0.67, val_acc : 0.71
Epoch 9, loss : 33.94, val_loss : 18.85, acc : 0.58, val_acc : 0.50
Epoch 10, loss : 22.39, val_loss : 24.64, acc : 0.50, val_acc : 0.58
Epoch 11, loss : 18.29, val_loss : 19.11, acc : 0.67, val_acc : 0.62
Epoch 12, loss : 21.23, val_loss : 12.95, acc : 0.58, val_acc : 0.54
Epoch 13, loss : 21.76, val_loss : 11.50, acc : 0.58, val_acc : 0.58
Epoch 14, loss : 19.76, val_l

## <font color="DeepPink">回帰問題ではaccuracyを見てはいけない（分類とは違い、当たることは無いので）。そのため、lossが下がっていることが確認できれば良い。<font>

# ---------MNIST---------
ニューラルネットワークをスクラッチで作成した際に使用したデータセットを使う。以前ダウンロードしたときのpathを指定すると良い。

In [29]:
from sklearn.datasets import fetch_mldata
from  sklearn.model_selection import train_test_split
import numpy as np

# 保存先を指定
mnist_dir = "./mnist_data/"

# MNISTの読み込み
mnist = fetch_mldata('MNIST original', data_home=mnist_dir)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

(44800, 784) (11200, 784) (14000, 784)
(44800,) (11200,) (14000,)


In [30]:
# ラベルをint型にしておく
y_train = y_train.astype(np.int)
y_test = y_test.astype(np.int)
y_val = y_val.astype(np.int)

# 特徴量だけを、float型へ変換している
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_val = X_val.astype(np.float)

# 最大値・最小値の確認（処理前）
print(X_train.max())
print(X_train.min())
print("----")

# 正則化？正規化？標準化？している
X_train /= 255
X_test /= 255
X_val /= 255

# 最大値・最小値の確認（処理前）
print(X_train.max())
print(X_train.min())

255.0
0.0
----
1.0
0.0


In [31]:
y_train.shape, y_test.shape, y_val.shape

((44800,), (14000,), (11200,))

In [32]:
#  one-hot表現にする
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

# 教師ラベルだけ変換する
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])
y_val_one_hot = enc.transform(y_val[:, np.newaxis])

# shapeの確認
print("y_train.shape    　　　　　　　　　　　　　　　　:   ",y_train.shape)
print("y_train_one_hot.shape    :   ",y_train_one_hot.shape)
print("y_train_one_hot.型  　　　　　　  :   ",y_train_one_hot.dtype)
print("------")
print("y_test.shape  　　　　　　　　　　　　　　　　  :   ",y_test.shape)
print("y_test_one_hot.shape    :   ",y_test_one_hot.shape)
print("y_test_one_hot.型　　　　　　    :   ",y_test_one_hot.dtype)
print("------")
print("y_val.shape  　　　　　　　　　　　　　　　　  :   ",y_val.shape)
print("y_val_one_hot.shape    :   ",y_val_one_hot.shape)
print("y_val_one_hot.型　　　　　　    :   ",y_val_one_hot.dtype)

y_train.shape    　　　　　　　　　　　　　　　　:    (44800,)
y_train_one_hot.shape    :    (44800, 10)
y_train_one_hot.型  　　　　　　  :    float64
------
y_test.shape  　　　　　　　　　　　　　　　　  :    (14000,)
y_test_one_hot.shape    :    (14000, 10)
y_test_one_hot.型　　　　　　    :    float64
------
y_val.shape  　　　　　　　　　　　　　　　　  :    (11200,)
y_val_one_hot.shape    :    (11200, 10)
y_val_one_hot.型　　　　　　    :    float64


In [42]:
import tensorflow as tf

class GetMiniBatch3:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._counter = 0
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __iter__(self):
        return self
    def __next__(self):
        if self._counter >= self._stop:
            self._counter = 0
            raise StopIteration()

        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.00005
batch_size = 200
num_epochs = 20

n_hidden1 = 400
n_hidden2 = 200
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train3 = GetMiniBatch3(X_train, y_train_one_hot, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    # 全結合
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    
    # 活性化関数relu
    layer_1 = tf.nn.relu(layer_1)
    
    # 全結合
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    
    # 活性化関数relu
    layer_2 = tf.nn.relu(layer_2)
    
    # 全結合のままで出力しているので、恒等関数で対応している意味合い？
    # →違う。出力層の活性化関数はsigmoidである。誤差逆伝搬のコードは★１
    # 誤差逆伝搬ではない、予測値を算出する処理は★2
    # つまり、誤差逆伝搬の「計算グラフ」では、★1と★２に枝分かれしている
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)
#print(tf.shape(logits))
# 目的関数
# reduce_meanは、np.meanと等価のようだ
# シグモイド＋交差エントロピーの組み合わせ.サンプル毎の損失をだすため、reduce_meanしている
# ★１
# ◼️◼️◼️◼️◼️ループで呼び出している箇所
#loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))

# 最適化手法
# AdamOptimizer　：Adamアルゴリズムを実装するオプティマイザ
# 重みを更新する処理
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

# ◼️◼️◼️◼️◼️ループで呼び出している箇所
# loss更新によって最小化する操作を追加し、var_list
train_op = optimizer.minimize(loss_op)

# 推定結果 -0.5は閾値　★2
#tmp1 = tf.sign(Y - 0.5) # debug
#tmp2 = tf.sigmoid(logits)
#correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
#correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.nn.softmax(logits) - 0.5))
correct_pred = tf.equal(tf.argmax(Y,1), tf.argmax(logits,1))
#a = np.argmax(Y)
# 指標値計算
# ◼️◼️◼️◼️◼️ループで呼び出している箇所
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    # このrunでは、initだけを実行している。
    sess.run(init)
    for epoch in range(num_epochs):
        
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        #print(X_train.shape[])
        total_loss = 0
        total_acc = 0
        
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train3):
            
            # ミニバッチごとにループ
            #print(mini_batch_x.shape)
            #print(mini_batch_y.shape)
            #print(mini_batch_x.shape)
            #a = sess.run(a, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            #print(a)
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            #aa  = sess.run(correct_pred, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            #bb = sess.run(correct_pred, feed_dict={X: mini_batch_x, Y: mini_batch_y}) #debug
            #print(aa)
            #cc = sess.run(tmp2, feed_dict={X: mini_batch_x, Y: mini_batch_y})  #debug
            #print("correct_pred", bb.dtype)
            #print("tf.sigmoid(logits).shape",cc.shape)
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val_one_hot})
        print("Epoch {}, loss : {:.1f}, val_loss : {:.1f}, acc : {:.2f}, val_acc : {:.2f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test_one_hot})
    print("test_acc : {:.3f}".format(test_acc))
    

Epoch 0, loss : 1059.4, val_loss : 1097.4, acc : 0.19, val_acc : 0.18
Epoch 1, loss : 698.0, val_loss : 688.1, acc : 0.25, val_acc : 0.26
Epoch 2, loss : 488.5, val_loss : 477.3, acc : 0.38, val_acc : 0.37
Epoch 3, loss : 374.2, val_loss : 363.0, acc : 0.46, val_acc : 0.46
Epoch 4, loss : 297.2, val_loss : 289.2, acc : 0.54, val_acc : 0.53
Epoch 5, loss : 243.6, val_loss : 239.7, acc : 0.58, val_acc : 0.59
Epoch 6, loss : 203.9, val_loss : 205.0, acc : 0.61, val_acc : 0.63
Epoch 7, loss : 175.2, val_loss : 179.6, acc : 0.64, val_acc : 0.67
Epoch 8, loss : 152.7, val_loss : 160.2, acc : 0.68, val_acc : 0.69
Epoch 9, loss : 136.2, val_loss : 144.9, acc : 0.70, val_acc : 0.72
Epoch 10, loss : 124.2, val_loss : 132.6, acc : 0.73, val_acc : 0.74
Epoch 11, loss : 113.7, val_loss : 122.5, acc : 0.74, val_acc : 0.75
Epoch 12, loss : 104.4, val_loss : 114.1, acc : 0.76, val_acc : 0.76
Epoch 13, loss : 96.3, val_loss : 107.0, acc : 0.76, val_acc : 0.78
Epoch 14, loss : 88.5, val_loss : 100.9, ac

# ---------House Prices---------
この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使う。説明変数はさらに増やしても良い。

前の2つと異なり回帰問題である。分類問題と回帰問題の違いを理解している必要がある。

In [43]:
import pandas as pd

data = pd.read_csv("/Users/tsuneo/kaggle/houseprice/train.csv")
X = data[["GrLivArea", "YearBuilt"]].values
y = data["SalePrice"].values

#print(X.shape)
y = y[:, np.newaxis]
# 正規化
X = np.log(X)
y = np.log(y)

from  sklearn.model_selection import train_test_split

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

(934, 2) (234, 2) (292, 2)
(934, 1) (234, 1) (292, 1)


In [53]:
class GetMiniBatch4:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._counter = 0
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __iter__(self):
        return self
    def __next__(self):
        if self._counter >= self._stop:
            self._counter = 0
            raise StopIteration()

        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.0001
batch_size = 20
num_epochs = 20

n_hidden1 = 400
n_hidden2 = 200
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train4 = GetMiniBatch4(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    # 全結合層っぽい
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    
    # 活性化関数reluのようだ
    layer_1 = tf.nn.relu(layer_1)
    
    # 全結合
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    
    # 活性化関数relu
    layer_2 = tf.nn.relu(layer_2)
    
    # 全結合のままで出力しているので、恒等関数で対応している意味合い？
    # →違う。出力層の活性化関数はsigmoidである。誤差逆伝搬のコードは★１
    # 誤差逆伝搬ではない、予測値を算出する処理は★2
    # つまり、誤差逆伝搬の「計算グラフ」では、★1と★２に枝分かれしている
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
# reduce_meanは、np.meanと等価のようだ
# シグモイド＋交差エントロピーの組み合わせ.サンプル毎の損失をだすため、reduce_meanしている
# ★１
# ◼️◼️◼️◼️◼️ループで呼び出している箇所
#loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
#loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 出力層に活性化関数は通さない（恒等関数）
loss_op = tf.reduce_mean(tf.losses.mean_squared_error(labels=Y, predictions=logits))

# 最適化手法
# AdamOptimizer　：Adamアルゴリズムを実装するオプティマイザ
# 重みを更新する処理
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

# ◼️◼️◼️◼️◼️ループで呼び出している箇所
# loss更新によって最小化する操作を追加し、var_list
train_op = optimizer.minimize(loss_op)


# 推定結果 -0.5は閾値　★2
#tmp1 = tf.sign(Y - 0.5) # debug
#tmp2 = tf.sigmoid(logits)
#correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
#correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.nn.softmax(logits) - 0.5))
correct_pred = logits

# 指標値計算
# ◼️◼️◼️◼️◼️ループで呼び出している箇所
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    # このrunでは、initだけを実行している。
    sess.run(init)
    for epoch in range(num_epochs):
        
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        #print(X_train.shape[])
        total_loss = 0
        total_acc = 0
        
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train4):
            
            # ミニバッチごとにループ
            #print(mini_batch_x.shape)
            #print(mini_batch_y.shape)
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            #aa  = sess.run(correct_pred, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            #bb = sess.run(correct_pred, feed_dict={X: mini_batch_x, Y: mini_batch_y}) #debug
            #print(acc)
            #cc = sess.run(tmp2, feed_dict={X: mini_batch_x, Y: mini_batch_y})  #debug
            #print("correct_pred", bb.dtype)
            #print("tf.sigmoid(logits).shape",cc.shape)
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.2f}, val_loss : {:.2f}, acc : {:.2f}, val_acc : {:.2f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))
    

Epoch 0, loss : 390.79, val_loss : 397.32, acc : 0.33, val_acc : -0.11
Epoch 1, loss : 244.60, val_loss : 240.35, acc : 15.48, val_acc : 15.15
Epoch 2, loss : 170.14, val_loss : 172.10, acc : 11.62, val_acc : 11.23
Epoch 3, loss : 125.59, val_loss : 124.08, acc : 12.37, val_acc : 12.00
Epoch 4, loss : 92.38, val_loss : 86.90, acc : 12.64, val_acc : 12.28
Epoch 5, loss : 63.39, val_loss : 56.93, acc : 13.01, val_acc : 12.60
Epoch 6, loss : 42.24, val_loss : 33.20, acc : 13.05, val_acc : 12.80
Epoch 7, loss : 23.03, val_loss : 17.58, acc : 13.41, val_acc : 12.90
Epoch 8, loss : 12.88, val_loss : 10.50, acc : 13.48, val_acc : 12.75
Epoch 9, loss : 9.68, val_loss : 8.50, acc : 13.37, val_acc : 12.56
Epoch 10, loss : 8.59, val_loss : 7.97, acc : 13.35, val_acc : 12.51
Epoch 11, loss : 7.94, val_loss : 7.62, acc : 13.29, val_acc : 12.45
Epoch 12, loss : 7.52, val_loss : 7.27, acc : 13.26, val_acc : 12.43
Epoch 13, loss : 7.13, val_loss : 6.89, acc : 13.24, val_acc : 12.42
Epoch 14, loss : 6.

## <font color="DeepPink">回帰問題ではaccuracyを見てはいけない（分類とは違い、当たることは無いので）。そのため、lossが下がっていることが確認できれば良い。<font>

# 5.（オプション）他のフレームワークやAPIでの実装

**Keras**  
Kerasには2種類のAPI（記述方法）がある。Kerasのドキュメントは日本語が充実しているので読み進めやすい。
「Sequential API」

**Chainer**

**get_mnist()**

※今回は実装しませんでした。